### Cal-CRAI Reprojection -- LARGE geospatial file inputs
This notebook processes geospatial data files, of large complex size, for reprojection where necessary and includes the necessary metadata pieces for clarity. 

In [ ]:
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import boto3
import sys
import xarray as xr
import pandas as pd
from functools import wraps
import dask_geopandas
import re
sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
s3_client = boto3.client('s3')
def list_geospatial_files(path):
    """ Build a list of shapefile URIs contained in S3 folder """
    # initiate empty list for s3 URIs
    all_shapefiles = []
    bucket_name = 'ca-climate-index' 
    # initiate s3 session
    session = boto3.Session()
    # use the session to get the resource
    s3 = session.resource('s3')
    my_bucket = s3.Bucket(bucket_name)
    # iterate through directory
    for obj in my_bucket.objects.filter(
        Prefix=path):
        # build list of shapefile URIs
        if obj.key.endswith('.zip'):
            # preceding the URI with 'zip' lets you read in the file without downloading, unzipping, etc
            s3_uri = f"zip+s3://ca-climate-index/"+obj.key
            all_shapefiles.append(s3_uri)
        elif obj.key.endswith('.shp'):
            s3_uri = "s3://ca-climate-index/"+obj.key
            all_shapefiles.append(s3_uri)
    return all_shapefiles

@append_metadata
def reproject_large_shapefile(shp_fname, ca_boundaries, run_code=True, varname=''):
    """
    Reprojects large shapefiles to California Census Tract Coordinate Reference System, then clips to these 
    CA tracts, and uploads to AWS S3. This code differs from the 
    reproject_shapefile() function by utilizing dask-geopandas to manipulate large datasets. 

    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in
    ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
     
    Methods
    -------
    Use dask-geopandas to work with the large datasets
    
    Parameters
    ----------
    shp_fname: string
        filename of the large geodataframe shapefile
    ca_boundaries: 
        read-in gpd file of California Census Tracts
    run_code: bool
        if True, code will run. If false, just metadata file will be updated

    Script
    ------
    large_geospatial_reproject.ipynb    
    """
    s3_client = boto3.client('s3')  
    bucket_name = 'ca-climate-index' 

    if shp_fname.endswith('.zip'):
        shp_fname_intermediate = shp_fname.replace(
            'zip+',
            '')
    else:
        shp_fname_intermediate = shp_fname
                
    print('Data transformation: Reproject to standard coordinate reference system: 4269.')
    if varname == 'governance_usda_fuel_reduction':
        print("Data transformation: Subsetted from countrywide to California.")
    
    print('Data transformation: sjoin large geodata with CA census tract boundaries data.')
    
    if varname=="climate_iowa_mesonet_flash_flood_warnings":
        print(
                "Data transformation: Saved as 10 spatially coherent parquet files because"
                +" the resulting dataset is too large to be saved as one file."
        )
        print(f"Database saved to: s3://ca-climate-index/2b_reproject/climate_risk/flood/exposure/isu_environmental_mesonet/")
    else:
        dest_f = shp_fname_intermediate.replace(
            shp_fname_intermediate.split('/')[-1],f"{varname}.parquet.gzip")
        dest_f = re.sub(r'1_pull_data|2a_subset', '2b_reproject', dest_f)
        print('Data transformation: Saved as a parquet file')
        print(f"Database saved to: {dest_f}")
        
    if run_code==True:
        # read in shapefile of interest from S3 
        gdf = dask_geopandas.read_file(shp_fname, npartitions=10)
        orig_crs = gdf.crs

        if (orig_crs==ca_boundaries.crs):   
            print(f"Do not need to reproject {varname} since it is already in the same projection as the Census Tracts Shapefile.")
        else:       
            gdf = gdf.to_crs(ca_boundaries.crs)

        # one file needs to be subsetted to California first
        if varname == 'governance_usda_fuel_reduction':         
            df_list = []
            gdf = gdf.drop(columns=['REV_DATE'])
            for i in range(len(list(gdf.partitions))):
                edf = gdf.partitions[i].compute()
                cdf = edf.loc[edf['STATE_ABBR']=='CA']
            df_list.append(cdf) 
            all_ca_df = pd.concat(df_list)
            clipped_gdf = all_ca_df.sjoin(ca_boundaries, how='inner', predicate='intersects')            
        else:       
            # shuffle the geodataframes into spatially coherent partitions
            ddf = gdf.spatial_shuffle()       
            ca_ddf = dask_geopandas.from_geopandas(ca_boundaries, npartitions=10)
            ca_ddf = ca_ddf.spatial_shuffle()      
            clipped_gdf = ddf.sjoin(ca_ddf, how='inner', predicate='intersects').compute()
            # try to minimize the data by dropping unnecessary columns
        
        clipped_gdf = clipped_gdf.reset_index()
        to_drop=['USCB_STATEFP', 'USCB_COUNTYFP', 'USCB_TRACTCE', 
        'USCB_NAMELSAD', 'USCB_MTFCC', 'USCB_FUNCSTAT',
        'USCB_ALAND', 'USCB_AWATER', 'USCB_INTPTLAT', 'USCB_INTPTLON']
        clipped_gdf = clipped_gdf.drop(columns=to_drop)        
        ddf_part = dask_geopandas.from_geopandas(clipped_gdf, npartitions=10)
        
        if varname=="climate_iowa_mesonet_flash_flood_warnings":
            for i in range(len(list(ddf_part.partitions))):
                df = ddf_part.partitions[i].compute()
                # upload it to s3
                dest_f = shp_fname_intermediate.replace(
                    shp_fname_intermediate.split('/')[-1],f"{varname}_{i}.parquet.gzip")
                df.to_parquet(dest_f, compression='gzip')
        else:
            # upload it to s3
            ddf_part.to_parquet(dest_f, compression='gzip')


In [ ]:
# read in the CSV with the data details
ref_file = sys.path[-1]+'/metadata/Full Data Pipeline Notes - 1_ Pull.csv'
df = pd.read_csv(ref_file)

# subset for shapefiles
ref_df = df.fillna('N/A')
# comment out for now as 'Pulled Format' column not updated
ref_df = ref_df[
(ref_df["Pulled Format"].str.contains("shp")) 
| (ref_df["Pulled Format"].str.contains("gdb"))
]

### Define the path
path1 = "1_pull_data"
path2 = "2a_subset"
#  build a list of shapefiles in the above s3 paths
my_list = list_geospatial_files(path1) 
my_list += list_geospatial_files(path2)

# read in CA census tiger file
census_shp_dir = "s3://ca-climate-index/0_map_data/2021_tiger_census_tract/2021_ca_tract/"
ca_boundaries = gpd.read_file(census_shp_dir)
# need to rename columns so we don't have any duplicates in the final geodatabase
column_names = ca_boundaries.columns
new_column_names = ["USCB_"+column for column in column_names if column != "geometry"]
ca_boundaries = ca_boundaries.rename(columns=dict(zip(column_names, new_column_names)))

In [ ]:
large_files = [
    'climate_iowa_mesonet_flash_flood_warnings', 
    'climate_koordinates_floodplain', 
    'climate_iowa_mesonet_wildfire_warnings',
    'governance_usda_watershed_risk',
    'governance_usda_fuel_reduction'
]

# get reference dataframe for the above large files
large_df = ref_df[ref_df.Variable.isin(large_files)]
# build list of paths corresponding to the large files
large_list = [f for f in my_list if f.split('/')[-1] in large_df["File Name"].values]
# dictionary to map file names to variable names
fname_dict = dict(zip(large_df["File Name"].values, large_df["Variable"].values))

In [ ]:
for fpath in large_list:
    # get the file name by itself (no subdirectories)
    fname = fpath.split('/')[-1]
    varname = fname_dict[fname]
    print(varname)
    reproject_large_shapefile(fpath, ca_boundaries, run_code=False, varname=varname)